In [2]:

# %%
# ========== 第一部分：环境初始化和地图创建 ==========
# 导入Google Earth Engine (GEE) Python API，用于访问和处理地理空间数据
import ee
# 导入geemap库，提供交互式地图界面和GEE的高级功能
import geemap
ee.Authenticate()
ee.Initialize(project='gee-ee-20031001')

# %%
# 创建一个交互式地图实例
# center参数设置地图初始中心点为[纬度2.5, 经度105]（东南亚地区）
# zoom参数设置初始缩放级别为5（数值越大缩放越近）
Map = geemap.Map(center=[2.5, 105], zoom=5) 

# %%
# ========== 第二部分：研究区域定义 ==========
# 定义研究区域为一个矩形范围
# ee.Geometry.Rectangle([最小经度, 最小纬度, 最大经度, 最大纬度])
# 这里定义的是东经100°-110°，纬度-5°-10°之间的东南亚区域
region = ee.Geometry.Rectangle([100, -5, 110, 10])  # 东南亚区域

# %%
# ========== 第三部分：森林覆盖数据加载和处理 ==========
# 从GEE数据目录加载2020年JRC全球森林覆盖数据集
# "JRC/GFC2020/V2"是欧盟联合研究中心提供的全球森林覆盖数据
# 该数据集包含2020年全球森林覆盖情况，像元值为1表示森林，0表示非森林
forest2020 = ee.ImageCollection("JRC/GFC2020/V2").mosaic()
forestMask = forest2020.eq(1)

# %%
# 使用updateMask()函数将森林掩膜应用到原始影像
# 这一步会保留森林区域（掩膜值为True的部分），非森林区域会被设置为无效值
# clip()函数将处理后的影像裁剪到我们定义的研究区域
# 最终forestOnly只包含研究区域内的森林像元
forestOnly = forest2020.updateMask(forestMask).clip(region)

# %%
# ========== 第四部分：NDVI数据获取和处理 ==========
# 加载MODIS MOD13A2数据集，这是NASA提供的植被指数产品
# 空间分辨率为1公里，时间分辨率为16天，包含NDVI波段
# 筛选2023年1月1日至12月31日的所有影像
ndviCollection = ee.ImageCollection("MODIS/006/MOD13A2") \
    .filterDate('2023-01-01', '2023-12-31') \
    .select('NDVI') \
    .filterBounds(region)

# %%
# 计算2023年全年NDVI数据的平均值
# mean()函数计算影像集合中所有影像的逐像元平均值
# clip()函数将结果裁剪到研究区域
ndviMean2023 = ndviCollection.mean().clip(region)

# %%
# 使用森林掩膜对NDVI均值数据进行掩膜处理
# 这样只保留森林区域的NDVI值，非森林区域的NDVI值被排除
# 最终ndviInForest只包含森林区域的NDVI信息
ndviInForest = ndviMean2023.updateMask(forestMask)

# %%
# ========== 第五部分：退化森林识别 ==========
# 识别潜在退化森林区域
# 设定NDVI阈值4000作为森林活力标准
# lt(4000)函数创建布尔影像，NDVI小于4000的区域标记为True（潜在退化）
# 这个阈值可以根据具体研究需求进行调整
degradedForest = ndviInForest.lt(4000)

# %%
# ========== 第六部分：可视化参数设置 ==========
# 设置森林覆盖图层的可视化参数
# min和max设置为1，因为我们只显示森林像元（值为1）
# palette设置颜色为深绿色('#4d9221')
forestVis = {'min': 1, 'max': 1, 'palette': ['#4d9221']}

# %%
# 设置NDVI图层的可视化参数
# min=0, max=9000对应实际NDVI值0到0.9
# palette使用渐变色从浅黄到深绿，表示NDVI从低到高
ndviVis = {'min': 0, 'max': 9000, 'palette': ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']}

# %%
# 设置退化森林图层的可视化参数
# 使用红色('#ff0000')突出显示潜在退化森林区域
degradedVis = {'palette': ['#ff0000']}

# %%
# ========== 第七部分：地图图层添加 ==========
Map.addLayer(forestOnly, forestVis, '2020年森林覆盖')
Map.addLayer(ndviInForest, ndviVis, '2023年森林NDVI')
Map.addLayer(degradedForest.updateMask(degradedForest), degradedVis, '潜在退化森林区域')

# %%
# ========== 第八部分：图例添加 ==========
# 添加森林覆盖图例
# 位置设置在地图右上角，只包含森林一类
Map.add_legend(
    title='森林覆盖',
    keys=['森林'],
    colors=['#4d9221'],
    position='topright'
)

# %%
# 添加NDVI指数图例
# 显示NDVI值的分级，位置在左下角
Map.add_legend(
    title='NDVI指数',
    keys=['0', '2000', '4000', '6000', '9000'],
    colors=['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837'],
    position='bottomleft'
)

# %%
# 添加潜在退化森林图例
# 解释红色区域代表NDVI小于4000的潜在退化森林
Map.add_legend(
    title='潜在退化森林',
    keys=['NDVI < 4000'],
    colors=['#ff0000'],
    position='bottomright'
)

# %%
# ========== 第九部分：显示地图 ==========
Map


Map(center=[2.5, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…